In [ ]:
import psycopg2
import json
import folium

def show_multiple_layers_map(layer_ids, dbname, user, password, host, port):
    """
    Recupera e visualizza i GeoJSON di più ACE (layer_id) da Postgres su una mappa Folium.
    """
    # Connessione al DB
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cursor = conn.cursor()

    try:
        geojson_features = []

        for layer_id in layer_ids:
            sql = """
            SELECT geojson
            FROM geom_data
            WHERE layerid = %s
            """
            cursor.execute(sql, (layer_id,))
            result = cursor.fetchone()

            if not result:
                print(f"[!] Nessun record trovato per layerid = {layer_id}")
                continue

            geojson_str = result[0]
            if isinstance(geojson_str, dict):
                geojson_data = geojson_str
            else:
                geojson_data = json.loads(geojson_str)

            # Aggiunge il Feature al gruppo
            geojson_features.append(geojson_data)

        if not geojson_features:
            print("Nessuna geometria valida trovata.")
            return

        # Prendiamo il centro della prima geometria per inizializzare la mappa
        geometry = geojson_features[0]['geometry']
        coords = geometry['coordinates'][0][0][0]
        center_lon, center_lat = coords[0], coords[1]

        m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

        # Aggiungiamo ogni geometria come un layer separato
        for i, feature in enumerate(geojson_features):
            folium.GeoJson(
                feature,
                name=f"Layer {layer_ids[i]}",
                style_function=lambda x: {
                    'fillColor': 'red',
                    'color': 'red',
                    'weight': 2,
                    'fillOpacity': 0.5
                }
            ).add_to(m)


        folium.LayerControl().add_to(m)

        return m  # Mostra direttamente in Jupyter

    finally:
        cursor.close()
        conn.close()


In [ ]:
# Visualizza 10 ACE
worst = ['08|033|047|000|000', '08|034|022|000|000', '08|033|015|000|000', '08|040|031|000|000', '08|033|030|000|000', '08|039|007|000|001', '08|033|017|000|000', '08|034|001|000|000', '08|035|019|000|000', '08|099|002|000|000']
best = ['08|037|057|000|000', '08|035|033|003|031', '08|037|053|000|001', '08|037|006|001|000', '08|035|012|000|000', '08|034|008|000|000', '08|035|033|002|000', '08|099|004|000|000', '08|034|025|000|000', '08|036|007|000|000']
m = show_multiple_layers_map(
    layer_ids= worst,
    dbname='',
    user='',
    password=''
)
m
